In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [ ]:
! pip install --upgrade jupyter ipywidgets

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from ragchecker.integrations.llama_index import response_to_rag_results
from ragchecker import RAGResults, RAGChecker
from ragchecker.metrics import all_metrics

In [ ]:
! pip install llama-index-readers-web


<hr>

Scrap the web page for answer the certain question

In [6]:

from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display

In [7]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://www.ml.school/"]
)

In [8]:
documents[0]

Document(id_='http://www.ml.school/', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='> "This is the best machine learning course I\'ve done. Worth every cent."\n\n**Jose Reyes** , AI/ML at Cevo Australia\n\n# Building Machine Learning Systems That Don\'t Suck\n\nA **live, interactive program** that\'ll help you build **production-ready**\nmachine learning systems from the ground up.\n\nNext cohort: November 4 \\- 21, 2024\n\nCheck the schedule for more details about upcoming cohorts.\n\nI want to join!Sign in\n\n## Learn how to **design** , **build** , **deploy** , and **scale** machine\nlearning systems to solve real-world problems.\n\nI\'ll lose my mind if I see another book or course teaching people the same\nbasic ideas for the hundredth time. Most people are stuck in beginner mode,\nand finding help to solve real-world problems is hard.\n\nI want to change that.\n\nI started writing software 30 years ago. I\'ve 

In [9]:
index = SummaryIndex.from_documents(documents)

In [13]:
# Create query engine
rag_application = index.as_query_engine()

the query engine is working
<hr>

Using the original method to get the rag_results for evaluation

- But it failed
- successfully got the response and transform it to `rag_result`
- but failed to create `rag_results`

In [44]:
# User query and groud truth answer
user_query = "What is Price of this course?"
gt_answer = "The price of the course is $500"


# Get response from LlamaIndex
response_object = rag_application.query(user_query)

# Convert to RAGChecker format
rag_result = response_to_rag_results(
    query=user_query,
    gt_answer=gt_answer,
    response_object=response_object,
)

# Create RAGResults object
rag_results = RAGResults.from_dict({"results": [rag_result]})
print(rag_results)

RAGResults(
  1 RAG results,
  Metrics:
  {
    "overall_metrics": {},
    "retriever_metrics": {},
    "generator_metrics": {}
  }
)


Create a json file for the rag_results
- But it failed
- Generate the json file for the `rag_result`


In [49]:
with open("rag_response.json") as fp:
    rag_results = RAGResults.from_json(fp.read())


In [48]:
print(rag_results)

RAGResults(
  1 RAG results,
  Metrics:
  {
    "overall_metrics": {},
    "retriever_metrics": {},
    "generator_metrics": {}
  }
)


No matter what method I use, it failed

In [45]:
# Initialize RAGChecker
evaluator = RAGChecker(
    extractor_name="bedrock/meta.llama3-70b-instruct-v1:0",
    checker_name="bedrock/meta.llama3-70b-instruct-v1:0",
    batch_size_extractor=32,
    batch_size_checker=32,
)

# Evaluate using RAGChecker
evaluator.evaluate(rag_results, all_metrics)

# Print detailed results
print(rag_results)

2024-10-08 12:28:03.162 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for response of 1 RAG results.
  0%|          | 0/1 [00:01<?, ?it/s]

Exit with the following error:
'NoCredentialsError' object has no attribute 'response'


TypeError: 'NoneType' object is not iterable

Clear to see that the evaluation is not working

In [50]:
evaluator = RAGChecker(
    extractor_name="bedrock/meta.llama3-70b-instruct-v1:0",
    checker_name="bedrock/meta.llama3-70b-instruct-v1:0",
    batch_size_extractor=32,
    batch_size_checker=32,
)

# Check if rag_results contains valid data
if rag_results.results and all(result.response for result in rag_results.results):
    try:
        evaluator.evaluate(rag_results, all_metrics)
    except Exception as e:
        print(f"An error occurred during evaluation: {e}")
else:
    print("rag_results does not contain valid data for evaluation")

# Print detailed results
print(rag_results)

2024-10-08 12:32:44.245 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for response of 1 RAG results.
  0%|          | 0/1 [00:02<?, ?it/s]

Exit with the following error:
'NoCredentialsError' object has no attribute 'response'
An error occurred during evaluation: 'NoneType' object is not iterable
RAGResults(
  1 RAG results,
  Metrics:
  {
    "overall_metrics": {},
    "retriever_metrics": {},
    "generator_metrics": {}
  }
)
